<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>To get AWS ELB unhealthy instances using unSkript actions.</strong></div>
</center><center>
<h2 id="Get-AWS-ELB-Unhealthy-Instances">Get AWS ELB Unhealthy Instances</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1. Get Unhealthy instances from ELB<br>2. Post Slack Message<code>
</code></p>

In [19]:
if elb_name and not region:
    raise SystemExit("Provide region for the ELB instances!")
if region == None:
    region = ""
if elb_name == None:
    elb_name = ""

<h3 id="Get-Unhealthy-instances-from-ELB">Get Unhealthy instances from ELB</h3>
<p>Here we will use unSkript&nbsp;<strong>Get Unhealthy instances from ELB</strong> action. This action is used to get all unhealthy instances from ELB, the instances which are out of service are considered unhealthy instances.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>elb_name</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable</strong>: <code>unhealthy_instances</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions


from beartype import beartype
@beartype
def aws_get_unhealthy_instances_from_elb_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def aws_get_unhealthy_instances_from_elb(handle, elb_name: str = "", region: str = "") -> Tuple:
    """aws_get_unhealthy_instances_from_elb gives unhealthy instances from ELB

        :type elb_name: string
        :param elb_name: Name of the elastic load balancer.

        :type region: string
        :param region: AWS region.

        :rtype: A tuple with execution results and a list of unhealthy instances from ELB
    """

    result = []
    all_regions = [region]
    elb_list = []
    if not region:
        all_regions = aws_list_all_regions(handle)

    if not elb_name:
        for reg in all_regions:
            try:
                asg_client = handle.client('elb', region_name=reg)
                response = aws_get_paginator(
                    asg_client,
                    "describe_load_balancers",
                    "LoadBalancerDescriptions"
                    )
                for i in response:
                    elb_dict = {}
                    elb_dict["load_balancer_name"] = i["LoadBalancerName"]
                    elb_dict["region"] = reg
                    elb_list.append(elb_dict)
            except Exception:
                pass

    if elb_name and not region:
        for reg in all_regions:
            try:
                asg_client = handle.client('elb', region_name=reg)
                response = aws_get_paginator(
                    asg_client,
                    "describe_load_balancers",
                    "LoadBalancerDescriptions"
                    )
                for i in response:
                    if elb_name in i["LoadBalancerName"]:
                        elb_dict = {}
                        elb_dict["load_balancer_name"] = i["LoadBalancerName"]
                        elb_dict["region"] = reg
                        elb_list.append(elb_dict)
            except Exception:
                pass

    if elb_name and region:
        try:
            elbClient = handle.client('elb', region_name=region)
            res = elbClient.describe_instance_health(LoadBalancerName=elb_name)
            for instance in res['InstanceStates']:
                data_dict = {}
                if instance['State'] == "OutOfService":
                    data_dict["instance_id"] = instance["InstanceId"]
                    data_dict["region"] = reg
                    data_dict["load_balancer_name"] = i["LoadBalancerName"]
                    result.append(data_dict)
        except Exception:
            pass

    for elb in elb_list:
        try:
            elbClient = handle.client('elb', region_name=elb["region"])
            res = elbClient.describe_instance_health(LoadBalancerName=elb["load_balancer_name"])
            for instance in res['InstanceStates']:
                data_dict = {}
                if instance['State'] == "OutOfService":
                    data_dict["instance_id"] = instance["InstanceId"]
                    data_dict["region"] = reg
                    data_dict["load_balancer_name"] = i["LoadBalancerName"]
                    result.append(data_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "elb_name": "elb_name",
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not elb_names",
    "condition_result": true
    }''')
task.configure(outputName="unhealthy_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_unhealthy_instances_from_elb, lego_printer=aws_get_unhealthy_instances_from_elb_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we modify the output from step 1A and step 1B and return a list of dictionary items for the unhealthy instances from ELB.</p>
<blockquote>
<p><strong>Output variable:</strong> elb_instance_list</p>
</blockquote>

In [21]:
elb_instance_list = []
try:
    if unhealthy_instances:
        if unhealthy_instances[0] == False:
            for instance in unhealthy_instances[1]:
                elb_instance_list.append(instance)
except Exception as e:
    raise Exception(e)

<h3 id="Post-Slack-Message">Post Slack Message</h3>
<p>Here we will use unSkript <strong>Post Slack Message</strong> action. This action takes channel: str and message: str as input. This input is used to post the message to the slack channel.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>message</code>, <code>channel</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable</strong>: <code>message_status</code></p>
</blockquote>

In [14]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Unhealthy instances for elb:{elb_instance_list}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "channel",
    "condition_result": true
    }''')
task.configure(outputName="message_status")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated the use of unSkript's AWS and slack legos to perform AWS action and this runbook fetches the unHealthy EC2 instances for Classic ELB and posts to a slack channel. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>